### Imports and Dependencies

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import io
import requests
import os

from scipy.stats import zscore
from sklearn import preprocessing, metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.preprocessing import LabelEncoder
from IPython.display import display

from imblearn.over_sampling import SMOTE

from matplotlib import pyplot
import matplotlib.pyplot as plot
import matplotlib.pyplot as plt

Panda Preferences

In [2]:
# Set options for displaying maximum columns and rows
pd.set_option('display.max_columns', None)  
#pd.set_option('display.max_rows', None)

**Read CSV**

In [3]:
# Read the CSV file into a DataFrame
df = pd.read_csv("../../../Datasets/Flows/iomt_flows.csv")

# Display the DataFrame
display(df)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (6,7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,flow_duration,fwd_pkts_tot,bwd_pkts_tot,fwd_data_pkts_tot,bwd_data_pkts_tot,fwd_pkts_per_sec,bwd_pkts_per_sec,flow_pkts_per_sec,down_up_ratio,fwd_header_size_tot,fwd_header_size_min,fwd_header_size_max,bwd_header_size_tot,bwd_header_size_min,bwd_header_size_max,flow_FIN_flag_count,flow_SYN_flag_count,flow_RST_flag_count,fwd_PSH_flag_count,bwd_PSH_flag_count,flow_ACK_flag_count,fwd_URG_flag_count,bwd_URG_flag_count,flow_CWR_flag_count,flow_ECE_flag_count,fwd_pkts_payload.min,fwd_pkts_payload.max,fwd_pkts_payload.tot,fwd_pkts_payload.avg,fwd_pkts_payload.std,bwd_pkts_payload.min,bwd_pkts_payload.max,bwd_pkts_payload.tot,bwd_pkts_payload.avg,bwd_pkts_payload.std,flow_pkts_payload.min,flow_pkts_payload.max,flow_pkts_payload.tot,flow_pkts_payload.avg,flow_pkts_payload.std,fwd_iat.min,fwd_iat.max,fwd_iat.tot,fwd_iat.avg,fwd_iat.std,bwd_iat.min,bwd_iat.max,bwd_iat.tot,bwd_iat.avg,bwd_iat.std,flow_iat.min,flow_iat.max,flow_iat.tot,flow_iat.avg,flow_iat.std,payload_bytes_per_second,fwd_subflow_pkts,bwd_subflow_pkts,fwd_subflow_bytes,bwd_subflow_bytes,fwd_bulk_bytes,bwd_bulk_bytes,fwd_bulk_packets,bwd_bulk_packets,fwd_bulk_rate,bwd_bulk_rate,active.min,active.max,active.tot,active.avg,active.std,idle.min,idle.max,idle.tot,idle.avg,idle.std,fwd_init_window_size,bwd_init_window_size,fwd_last_window_size,bwd_last_window_size,traffic,is_attack
0,10.10.10.252,56248,224.0.0.251,5353,udp,dns,-,-,-,S0,T,F,0,D,1,78,0,0,-,0.000000,1,0,1,0,0.000000,0.000000,0.000000,0.000000,8,8,8,0,0,0,0,0,0,0,0,0,0,0,0,0,50.0,50.0,50.0,50.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,50.0,50.0,50.0,50.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,1.0,0.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,arpspoofing,1
1,10.10.10.252,50214,224.0.0.251,5353,udp,dns,-,-,-,S0,T,F,0,D,1,74,0,0,-,0.000000,1,0,1,0,0.000000,0.000000,0.000000,0.000000,8,8,8,0,0,0,0,0,0,0,0,0,0,0,0,0,46.0,46.0,46.0,46.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,46.0,46.0,46.0,46.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,1.0,0.0,46.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,arpspoofing,1
2,10.10.10.252,58369,224.0.0.251,5353,udp,dns,-,-,-,S0,T,F,0,D,1,73,0,0,-,0.000000,1,0,1,0,0.000000,0.000000,0.000000,0.000000,8,8,8,0,0,0,0,0,0,0,0,0,0,0,0,0,45.0,45.0,45.0,45.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,45.0,45.0,45.0,45.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,1.0,0.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,arpspoofing,1
3,10.10.10.252,57377,224.0.0.251,5353,udp,dns,-,-,-,S0,T,F,0,D,1,74,0,0,-,0.000000,1,0,1,0,0.000000,0.000000,0.000000,0.000000,8,8,8,0,0,0,0,0,0,0,0,0,0,0,0,0,46.0,46.0,46.0,46.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,46.0,46.0,46.0,46.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,1.0,0.0,46.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,arpspoofing,1
4,10.10.10.252,12345,224.0.0.251,5353,udp,dns,-,-,-,S0,T,F,0,D,1,74,0,0,-,0.000000,1,0,1,0,0.000000,0.000000,0.000000,0.000000,8,8,8,0,0,0,0,0,0,

-----------------------------------------------------------

## DF Statistics and Info

In [4]:
def display_information_dataframe(df_cop):
    # Create a summary of data types, column names, and unique values
    summary_data = [{'Data Type': dtype, 'Column Name': col, 'Unique Values': df_cop[col].unique()} for col, dtype in df_cop.dtypes.iteritems()]
    
    # Create a DataFrame from the summary data
    summary_df = pd.DataFrame(summary_data)
    
    # Set display options to show all rows and columns
    pd.options.display.max_rows = None
    pd.options.display.max_columns = None
    
    # Return the summary DataFrame
    return summary_df

In [5]:
display_information_dataframe(df)

,Data Type,Column Name,Unique Values
0,object,id.orig_h,"[10.10.10.252, 10.10.10.247, ::1, 10.10.10.248..."
1,int64,id.orig_p,"[56248, 50214, 58369, 57377, 12345, 40674, 571..."
2,object,id.resp_h,"[224.0.0.251, 10.10.10.0, 10.10.10.249, 10.10...."
3,int64,id.resp_p,"[5353, 137, 57621, 22, 5683, 1883, 123, 53, 56..."
4,object,proto,"[udp, tcp, icmp]"
5,object,service,"[dns, -, mqtt, dhcp, http]"
6,object,duration,"[-, 1.999524, 1.162033, 0.196591, 3.001097, 0...."
7,object,orig_bytes,"[-, 90, 170, 579, 1608, 583, 580, 80, 88, 135,..."
8,object,resp_bytes,"[-, 0, 477, 94, 300, 1200, 509, 571, 400, 6, 4..."
9,object,conn_state,"[S0, OTH, SHR, SF, SH, REJ, RSTR, S1, RSTOS0, ..."


--------------------------------------------

## Pre-processing and Data Encoding

Remove Outliers

In [6]:
def remove_outliers(df,columns,n_std):
    for col in columns:
        print(f'[REMOVED OUTLIERS] {col}')
        
        mean = df[col].mean()
        sd = df[col].std()
        
        df = df[(df[col] <= mean+(n_std*sd))]
        
    return df

In [7]:
outliers = [
    'bwd_pkts_payload.avg',
    'bwd_header_size_tot',
    'fwd_pkts_tot',
    'fwd_header_size_tot',
    'flow_pkts_payload.tot',
]

df = remove_outliers(df, outliers, 3)

[REMOVED OUTLIERS] bwd_pkts_payload.avg
[REMOVED OUTLIERS] bwd_header_size_tot
[REMOVED OUTLIERS] fwd_pkts_tot
[REMOVED OUTLIERS] fwd_header_size_tot
[REMOVED OUTLIERS] flow_pkts_payload.tot


#### Maping

In [8]:
# Define a mapping of values to replace
service_map = {'-': 0, 'dns': 1, 'mqtt': 2, 'dhcp': 3, 'http': 4}
bytes_map = {'-': 0}
proto_map = {'tcp': 1, 'udp': 2, 'icmp': 3}

# Replace the values in the columns based on the mapping
df['service'].replace(service_map, inplace=True)
df['orig_bytes'].replace(bytes_map, inplace=True)
df['resp_bytes'].replace(bytes_map, inplace=True)
df['proto'].replace(proto_map, inplace=True)

In [9]:
df['service'] = df['service'].astype(str).astype(int)
df['orig_bytes'] = df['orig_bytes'].astype(str).astype(int)
df['resp_bytes'] = df['resp_bytes'].astype(str).astype(int)
df['proto'] = df['proto'].astype(str).astype(int)

#### Feature engineering: 

For Resp and Dest, bytes and pkts

In [10]:
# Total Bytes Exchanged
df['total_bytes'] = df['orig_bytes'] + df['resp_bytes']

# Total Packets Exchanged
df['total_pkts'] = df['orig_pkts'] + df['resp_pkts']

# Average Packet Size
df['avg_orig_pkt_size'] = df['orig_bytes'] / df['orig_pkts']
df['avg_resp_pkt_size'] = df['resp_bytes'] / df['resp_pkts']

# Packet Size per Packet Ratio
df['orig_size_per_pkt'] = df['orig_bytes'] / df['orig_pkts']
df['resp_size_per_pkt'] = df['resp_bytes'] / df['resp_pkts']

# Direction of Traffic
df['pkts_unidirectional_traffic'] = ((df['orig_pkts'] == 0) | (df['resp_pkts'] == 0)).astype(int)

# Traffic Anomalies
df['pkt_difference'] = df['orig_pkts'] - df['resp_pkts']
df['byte_difference'] = df['orig_bytes'] - df['resp_bytes']

For Packets

In [11]:
# Combine packet counts into a single column for total packets
df['total_pkts'] = df['fwd_pkts_tot'] + df['bwd_pkts_tot']

# Combine payload packet counts into a single column for total payload packets
df['total_data_pkts'] = df['fwd_data_pkts_tot'] + df['bwd_data_pkts_tot']

# Calculate the ratio of forward packets to backward packets
df['fwd_to_bwd_pkts_ratio'] = df['fwd_pkts_tot'] / df['bwd_pkts_tot']

# Calculate the ratio of payload packets to total packets
df['payload_ratio'] = df['total_data_pkts'] / df['total_pkts']

# Calculate the total payload volume in both directions
df['total_payload_volume'] = df['fwd_data_pkts_tot'] + df['bwd_data_pkts_tot']

# Calculate the ratio of payload volume in forward direction to backward direction
df['fwd_to_bwd_payload_volume_ratio'] = df['fwd_data_pkts_tot'] / df['bwd_data_pkts_tot']

For Packets - Per Sec

In [12]:
# Packet rate ratio
df['fwd_to_bwd_pkts_ratio'] = df['fwd_pkts_per_sec'] / df['bwd_pkts_per_sec']

# Packet rate difference
df['fwd_bwd_pkts_diff'] = df['fwd_pkts_per_sec'] - df['bwd_pkts_per_sec']

# Duration-weighted packet rate
df['duration_weighted_pkts'] = (df['fwd_pkts_per_sec'] + df['bwd_pkts_per_sec']) * df['flow_duration']

# Packet size weighted rate
df['pkts_size_weighted'] = (df['fwd_pkts_per_sec'] + df['bwd_pkts_per_sec']) * (df['fwd_pkts_payload.avg'] + df['bwd_pkts_payload.avg'])

# Flow packet size weighted rate
df['flow_pkts_size_weighted'] = df['flow_pkts_per_sec'] * df['flow_pkts_payload.avg']

For Header Size:

In [13]:
# Header Size Ratio
df['header_size_ratio'] = df['fwd_header_size_tot'] / (df['bwd_header_size_tot'] + 1)  # Adding 1 to avoid division by zero

# Total Header Size
df['total_header_size'] = df['fwd_header_size_tot'] + df['bwd_header_size_tot']

# Header Size Difference
df['header_size_diff'] = abs(df['fwd_header_size_tot'] - df['bwd_header_size_tot'])

For the Payload:

In [14]:
# Feature engineering functions
def calculate_ratio(x, y):
    return x / y if y != 0 else 0

def calculate_difference(x, y):
    return x - y

def calculate_range(x, y):
    return x - y

df['fwd_bwd_payload_ratio'] = df['fwd_pkts_payload.tot'] / df['bwd_pkts_payload.tot']
df['fwd_bwd_payload_avg_ratio'] = df['fwd_pkts_payload.avg'] / df['bwd_pkts_payload.avg']
df['fwd_bwd_payload_max_ratio'] = df['fwd_pkts_payload.max'] / df['bwd_pkts_payload.max']
df['fwd_bwd_payload_min_ratio'] = df['fwd_pkts_payload.min'] / df['bwd_pkts_payload.min']
df['fwd_bwd_payload_std_ratio'] = df['fwd_pkts_payload.std'] / df['bwd_pkts_payload.std']

df['fwd_bwd_payload_tot_diff'] = calculate_difference(df['fwd_pkts_payload.tot'], df['bwd_pkts_payload.tot'])
df['fwd_bwd_payload_avg_diff'] = calculate_difference(df['fwd_pkts_payload.avg'], df['bwd_pkts_payload.avg'])
df['fwd_bwd_payload_max_diff'] = calculate_difference(df['fwd_pkts_payload.max'], df['bwd_pkts_payload.max'])
df['fwd_bwd_payload_min_diff'] = calculate_difference(df['fwd_pkts_payload.min'], df['bwd_pkts_payload.min'])
df['fwd_bwd_payload_std_diff'] = calculate_difference(df['fwd_pkts_payload.std'], df['bwd_pkts_payload.std'])

df['flow_fwd_payload_ratio'] = df['flow_pkts_payload.tot'] / df['fwd_pkts_payload.tot']
df['flow_bwd_payload_ratio'] = df['flow_pkts_payload.tot'] / df['bwd_pkts_payload.tot']
df['flow_fwd_payload_diff'] = calculate_difference(df['flow_pkts_payload.tot'], df['fwd_pkts_payload.tot'])
df['flow_bwd_payload_diff'] = calculate_difference(df['flow_pkts_payload.tot'], df['bwd_pkts_payload.tot'])

df['flow_payload_range'] = calculate_range(df['flow_pkts_payload.max'], df['flow_pkts_payload.min'])
df['fwd_payload_cv'] = df['fwd_pkts_payload.std'] / df['fwd_pkts_payload.avg']
df['bwd_payload_cv'] = df['bwd_pkts_payload.std'] / df['bwd_pkts_payload.avg']
df['flow_payload_cv'] = df['flow_pkts_payload.std'] / df['flow_pkts_payload.avg']

For the interarrival time:

In [15]:
def calculate_packet_rate(num_packets, total_time):
    return num_packets / total_time

def calculate_interarrival_time_ratios(avg_fwd_iat, avg_bwd_iat, tot_fwd_iat, tot_bwd_iat):
    return avg_fwd_iat / avg_bwd_iat, tot_fwd_iat / tot_bwd_iat

# Packet rate features
df['fwd_iat_packet_rate'] = calculate_packet_rate(df['fwd_iat.tot'], df['fwd_iat.avg'])
df['bwd_iat_packet_rate'] = calculate_packet_rate(df['bwd_iat.tot'], df['bwd_iat.avg'])

# Flow features
df['flow_iat_packet_rate'] = calculate_packet_rate(df['flow_iat.tot'], df['flow_iat.avg'])

# Direction-based features
df['iat_is_unidirectional'] = np.isclose(df['bwd_iat.avg'], 0)

# Inter-arrival time ratios
df['avg_iat_ratio'], df['tot_iat_ratio'] = calculate_interarrival_time_ratios(
    df['fwd_iat.avg'], df['bwd_iat.avg'], df['fwd_iat.tot'], df['bwd_iat.tot']
)

For the active time:

In [16]:
# Convert time-based columns from microseconds to milliseconds
df['active.max'] /= 1000
df['active.min'] /= 1000
df['active.tot'] /= 1000
df['active.avg'] /= 1000
df['active.std'] /= 1000

df['active.tot_to_max_ratio'] = df['active.tot'] / df['active.max']
df['active.tot_to_min_ratio'] = df['active.tot'] / df['active.min']
df['avg_to_std_ratio'] = df['active.avg'] / df['active.std']

df['total_activity'] = df['active.tot'] * df['active.avg']

Split History:

In [17]:
def count_letters(string, is_upper):
    count = 0
    
    # Iterate through each character in the string
    for c in string:
        if is_upper and c.isupper():  # Check if the character is uppercase
            count += 1
        elif not is_upper and c.islower():  # Check if the character is lowercase
            count += 1
    
    return count

df['history_originator'] = df['history'].apply(lambda x: count_letters(x, True))
df['history_responder'] = df['history'].apply(lambda x: count_letters(x, False))

One Hot Encoding

In [18]:
def one_hot_encoding(df, columns):
    for col in columns:
        print(f'[ONE HOT ENCODING] {col}')
        df = pd.get_dummies(df, columns=[col], prefix=col)  # Perform one-hot encoding on the column
    
    return df

In [19]:
cols_to_encode = [
    'proto',
    'pkts_unidirectional_traffic',
    'iat_is_unidirectional',
    
]

df = one_hot_encoding(df,cols_to_encode)

[ONE HOT ENCODING] proto
[ONE HOT ENCODING] pkts_unidirectional_traffic
[ONE HOT ENCODING] iat_is_unidirectional


Normalize, Z-score

In [20]:
def zscore_normalization(df, cols):
    # Standardize the selected columns
    for col in cols:
        if col not in df.columns:
            print(f"[WARNING] {col} not found in DataFrame.")
            continue
        print(f"[Z-SCORE] {col}")
        df[col] = zscore(df[col])
    
    print("[DONE] Z-score Normalization")
    print("[INFO] Current Fields in the DataFrame:")
    return df

In [21]:
cols_to_zscore = [
    'fwd_data_pkts_tot',
    'bwd_data_pkts_tot',
    'fwd_pkts_per_sec',
    'bwd_pkts_per_sec',
    'flow_pkts_per_sec',
    'fwd_header_size_tot',
    'bwd_header_size_tot',
    'fwd_pkts_payload.tot',
    'fwd_pkts_payload.avg',
    'bwd_pkts_payload.tot',
    'bwd_pkts_payload.avg',
    'flow_pkts_payload.tot',
    'flow_pkts_payload.avg',
    'fwd_iat.tot',
    'fwd_iat.avg',
    'bwd_iat.tot',
    'bwd_iat.avg',
    'flow_iat.tot',
    'flow_iat.avg',
    'active.tot',
    'active.avg',
    'idle.tot',
    'idle.avg',
    
    'orig_bytes', 'resp_bytes',
    'orig_pkts', 'resp_pkts',
    
    'fwd_to_bwd_pkts_ratio',
    'payload_ratio',
    'total_payload_volume',
    'fwd_to_bwd_payload_volume_ratio',
    'fwd_bwd_pkts_diff',
    'flow_pkts_size_weighted',
    'pkts_size_weighted',
    'duration_weighted_pkts',
    'total_bytes',
    'avg_orig_pkt_size',
    'orig_size_per_pkt',
    'pkt_difference',
    'byte_difference',
    'header_size_ratio',
    'total_header_size',
    'header_size_diff',
    'fwd_bwd_payload_ratio',
    'fwd_bwd_payload_avg_ratio',
    'fwd_bwd_payload_max_ratio',
    'fwd_bwd_payload_min_ratio',
    'fwd_bwd_payload_std_ratio',
    'fwd_bwd_payload_tot_diff',
    'fwd_bwd_payload_avg_diff',
    'fwd_bwd_payload_max_diff',
    'fwd_bwd_payload_min_diff',
    'fwd_bwd_payload_std_diff',
    'flow_fwd_payload_ratio',
    'flow_bwd_payload_ratio',
    'flow_fwd_payload_diff',
    'flow_bwd_payload_diff',
    'flow_payload_range',
    'fwd_payload_cv',
    'bwd_payload_cv',
    'flow_payload_cv',
    'fwd_iat_packet_rate',
    'bwd_iat_packet_rate',
    'flow_iat_packet_rate',
    'total_activity',
]

df = zscore_normalization(df, cols_to_zscore)

[Z-SCORE] fwd_data_pkts_tot
[Z-SCORE] bwd_data_pkts_tot
[Z-SCORE] fwd_pkts_per_sec
[Z-SCORE] bwd_pkts_per_sec
[Z-SCORE] flow_pkts_per_sec
[Z-SCORE] fwd_header_size_tot
[Z-SCORE] bwd_header_size_tot
[Z-SCORE] fwd_pkts_payload.tot
[Z-SCORE] fwd_pkts_payload.avg
[Z-SCORE] bwd_pkts_payload.tot
[Z-SCORE] bwd_pkts_payload.avg
[Z-SCORE] flow_pkts_payload.tot
[Z-SCORE] flow_pkts_payload.avg
[Z-SCORE] fwd_iat.tot
[Z-SCORE] fwd_iat.avg
[Z-SCORE] bwd_iat.tot
[Z-SCORE] bwd_iat.avg
[Z-SCORE] flow_iat.tot
[Z-SCORE] flow_iat.avg
[Z-SCORE] active.tot
[Z-SCORE] active.avg
[Z-SCORE] idle.tot
[Z-SCORE] idle.avg
[Z-SCORE] orig_bytes
[Z-SCORE] resp_bytes
[Z-SCORE] orig_pkts
[Z-SCORE] resp_pkts
[Z-SCORE] fwd_to_bwd_pkts_ratio
[Z-SCORE] payload_ratio
[Z-SCORE] total_payload_volume
[Z-SCORE] fwd_to_bwd_payload_volume_ratio


/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py:2419: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd


[Z-SCORE] fwd_bwd_pkts_diff
[Z-SCORE] flow_pkts_size_weighted
[Z-SCORE] pkts_size_weighted
[Z-SCORE] duration_weighted_pkts
[Z-SCORE] total_bytes
[Z-SCORE] avg_orig_pkt_size
[Z-SCORE] orig_size_per_pkt
[Z-SCORE] pkt_difference
[Z-SCORE] byte_difference
[Z-SCORE] header_size_ratio
[Z-SCORE] total_header_size
[Z-SCORE] header_size_diff
[Z-SCORE] fwd_bwd_payload_ratio
[Z-SCORE] fwd_bwd_payload_avg_ratio
[Z-SCORE] fwd_bwd_payload_max_ratio
[Z-SCORE] fwd_bwd_payload_min_ratio
[Z-SCORE] fwd_bwd_payload_std_ratio
[Z-SCORE] fwd_bwd_payload_tot_diff
[Z-SCORE] fwd_bwd_payload_avg_diff
[Z-SCORE] fwd_bwd_payload_max_diff
[Z-SCORE] fwd_bwd_payload_min_diff
[Z-SCORE] fwd_bwd_payload_std_diff
[Z-SCORE] flow_fwd_payload_ratio
[Z-SCORE] flow_bwd_payload_ratio
[Z-SCORE] flow_fwd_payload_diff
[Z-SCORE] flow_bwd_payload_diff
[Z-SCORE] flow_payload_range
[Z-SCORE] fwd_payload_cv
[Z-SCORE] bwd_payload_cv
[Z-SCORE] flow_payload_cv
[Z-SCORE] fwd_iat_packet_rate
[Z-SCORE] bwd_iat_packet_rate
[Z-SCORE] flow_iat

Delete Insignificant Columns from the Dataframe

In [22]:
def delete_columns(df, cols):
    for col in cols:
        df.drop(col, axis = 1, inplace = True)
        print(f'[REMOVED] {col}')
    
    return df

In [23]:
cols_to_del = [
    'tunnel_parents',
    'missed_bytes',
    'idle.std',
    'idle.min',
    'idle.max',
    'fwd_URG_flag_count',
    'fwd_bulk_rate',
    'fwd_bulk_packets',
    'flow_ECE_flag_count',
    'flow_CWR_flag_count',
    'bwd_URG_flag_count',
    'bwd_pkts_payload.min',
    'bwd_iat.std',
    'bwd_bulk_rate',
    'bwd_bulk_packets',
    'bwd_bulk_bytes',
    'active.std',
    'id.orig_h',
    'id.orig_p',
    'id.resp_h',
    'id.resp_p',
    'history',
    'resp_ip_bytes',
    'fwd_PSH_flag_count',
    'flow_ACK_flag_count',
    'fwd_subflow_bytes',
    'bwd_subflow_bytes',
    'fwd_bulk_bytes',
    'conn_state',
    'flow_SYN_flag_count',
    'flow_RST_flag_count',
    'flow_FIN_flag_count',
    'fwd_pkts_payload.std',
    'bwd_iat.min',
    'bwd_iat.max',
    'payload_bytes_per_second',
    'flow_pkts_payload.std',
    'flow_pkts_payload.min',
    'flow_pkts_payload.max',
    'flow_iat.std',
    'flow_iat.min',
    'flow_iat.max',
    'bwd_PSH_flag_count',
    'fwd_subflow_pkts',
    'bwd_subflow_pkts',
    'orig_ip_bytes',
    'bwd_pkts_payload.std',
    'bwd_pkts_payload.max',
    'fwd_header_size_max',
    'fwd_header_size_min',
    'bwd_header_size_max',
    'bwd_header_size_min',
    'fwd_pkts_payload.min',
    'fwd_pkts_payload.max',
    'fwd_iat.std',
    'fwd_iat.min',
    'fwd_iat.max', 
    'active.min',
    'active.max',
    'local_orig',
    'local_resp',
    'fwd_pkts_tot',
    'bwd_pkts_tot',
    'fwd_init_window_size',
    'bwd_init_window_size',
    'fwd_last_window_size',
    'bwd_last_window_size',
    'idle.tot',
    'idle.avg',
    'duration',
    'flow_duration',

    'fwd_data_pkts_tot', 
    'bwd_data_pkts_tot',
    'fwd_pkts_per_sec', 
    'bwd_pkts_per_sec', 
    'flow_pkts_per_sec',
    'down_up_ratio',
    'fwd_header_size_tot',
    'bwd_header_size_tot',
    'fwd_pkts_payload.tot',
    'fwd_pkts_payload.avg',
    'bwd_pkts_payload.tot',
    'bwd_pkts_payload.avg',
    'flow_pkts_payload.tot',
    'flow_pkts_payload.avg',
    'fwd_iat.tot',
    'fwd_iat.avg',
    'bwd_iat.tot',
    'bwd_iat.avg',
    'flow_iat.tot',
    'flow_iat.avg',
    'active.tot',
    'active.avg',
    'orig_bytes',
    'resp_bytes',
    'orig_pkts',
    'resp_pkts',
    
    'avg_resp_pkt_size',
    'resp_size_per_pkt',
    'fwd_to_bwd_pkts_ratio',
    'avg_iat_ratio',
    'tot_iat_ratio',
    'active.tot_to_max_ratio',
    'active.tot_to_min_ratio',
    'avg_to_std_ratio',
    
    
    
    'avg_orig_pkt_size',
    'orig_size_per_pkt',
    'fwd_to_bwd_payload_volume_ratio',
    'fwd_bwd_payload_ratio',
    'fwd_bwd_payload_avg_ratio',
    'fwd_bwd_payload_max_ratio',
    'fwd_bwd_payload_min_ratio',
    'fwd_bwd_payload_std_ratio',
    'flow_fwd_payload_ratio',
    'flow_bwd_payload_ratio',
    'fwd_payload_cv',
    'bwd_payload_cv',
    'flow_payload_cv',
    'fwd_iat_packet_rate',
    'bwd_iat_packet_rate',
    'flow_iat_packet_rate',
    'fwd_bwd_payload_std_diff',
    'fwd_bwd_payload_min_diff',
    'fwd_bwd_payload_max_diff',
]

df = delete_columns(df,cols_to_del)

[REMOVED] tunnel_parents
[REMOVED] missed_bytes
[REMOVED] idle.std
[REMOVED] idle.min
[REMOVED] idle.max
[REMOVED] fwd_URG_flag_count
[REMOVED] fwd_bulk_rate
[REMOVED] fwd_bulk_packets
[REMOVED] flow_ECE_flag_count
[REMOVED] flow_CWR_flag_count
[REMOVED] bwd_URG_flag_count
[REMOVED] bwd_pkts_payload.min
[REMOVED] bwd_iat.std
[REMOVED] bwd_bulk_rate
[REMOVED] bwd_bulk_packets
[REMOVED] bwd_bulk_bytes
[REMOVED] active.std
[REMOVED] id.orig_h
[REMOVED] id.orig_p
[REMOVED] id.resp_h
[REMOVED] id.resp_p
[REMOVED] history
[REMOVED] resp_ip_bytes
[REMOVED] fwd_PSH_flag_count
[REMOVED] flow_ACK_flag_count
[REMOVED] fwd_subflow_bytes
[REMOVED] bwd_subflow_bytes
[REMOVED] fwd_bulk_bytes
[REMOVED] conn_state
[REMOVED] flow_SYN_flag_count
[REMOVED] flow_RST_flag_count
[REMOVED] flow_FIN_flag_count
[REMOVED] fwd_pkts_payload.std
[REMOVED] bwd_iat.min
[REMOVED] bwd_iat.max
[REMOVED] payload_bytes_per_second
[REMOVED] flow_pkts_payload.std
[REMOVED] flow_pkts_payload.min
[REMOVED] flow_pkts_payload.m

In [24]:
display_information_dataframe(df)

,Data Type,Column Name,Unique Values
0,int64,service,"[1, 0, 2, 3, 4]"
1,object,traffic,"[arpspoofing, normal, camoverflow, slowloris, ..."
2,int64,is_attack,"[1, 0]"
3,float64,total_bytes,"[-0.004437091619625206, -0.00442737194774535, ..."
4,int64,total_pkts,"[1, 10, 6, 2, 8, 7, 4, 3, 13, 18, 15, 5, 30, 2..."
5,float64,pkt_difference,"[-0.08365931844886242, -0.04441959626059166, 0..."
6,float64,byte_difference,"[-0.003765913566283288, -0.003753330639174888,..."
7,int64,total_data_pkts,"[1, 5, 6, 2, 4, 7, 10, 3, 13, 18, 15, 30, 24, ..."
8,float64,payload_ratio,"[2.503715535330146, 0.9727973936591321, 1.0905..."
9,float64,total_payload_volume,"[-0.14552838634335696, 0.9540779088977769, 1.2..."


In [25]:
df.groupby('duration_weighted_pkts')['duration_weighted_pkts'].count().sort_values(ascending=False)

duration_weighted_pkts
-0.463950     1955779
 0.026463       16603
 0.027363        8128
 0.027033        7121
 0.026971        7017
 0.026880        6827
 0.027363        6302
 0.026463        6201
 0.026463        6090
 0.027218        5990
 0.026921        5610
 0.027161        5448
 0.027927        4971
 0.027070        4777
 0.027218        4753
 0.027113        4752
 0.026792        4268
 0.027496        4234
 0.026846        4212
 0.027084        3637
 0.028595        3403
 0.027674        3306
 0.027578        3225
 0.027179        3218
 0.027459        3212
 0.027789        3210
 0.027578        3181
 0.027459        3102
 0.026945        3054
 0.026900        2933
 0.027033        2903
 0.027001        2876
 0.027322        2871
 0.027113        2841
 0.027058        2816
 0.027188        2809
 0.027309        2809
 0.027144        2781
 0.027284        2732
 0.027011        2709
 0.026752        2672
 0.027249        2661
 0.027578        2629
 0.027408        2590
 0.027284

In [26]:
# Separate the data into two dataframes for each class
apachekiller_df = df[df['traffic'] == 'apachekiller']
arpspoofing_df = df[df['traffic'] == 'arpspoofing']
camoverflow_df = df[df['traffic'] == 'camoverflow']
mqttmalaria_df = df[df['traffic'] == 'mqttmalaria']
netscan_df = df[df['traffic'] == 'netscan']
normal_df = df[df['traffic'] == 'normal']
rudeadyet_df = df[df['traffic'] == 'rudeadyet']
slowloris_df = df[df['traffic'] == 'slowloris']
slowread_df = df[df['traffic'] == 'slowread']

# Determine the proportion of attacks lines to keep
attack_num_lines_to_keep = 62500
normal_num_lines_to_keep = 300000

# Randomly select the lines to keep
camoverflow_subset = camoverflow_df.sample(n=attack_num_lines_to_keep, random_state=42)
netscan_subset = netscan_df.sample(n=attack_num_lines_to_keep, random_state=42)
normal_subset = normal_df.sample(n=normal_num_lines_to_keep, random_state=42)

# Combine the camoverflow subset with slowloris samples
df = pd.concat([apachekiller_df,arpspoofing_df,camoverflow_subset,mqttmalaria_df,netscan_subset,normal_subset,rudeadyet_df,slowloris_df,slowread_df], ignore_index=True)

# Shuffle the balanced dataset to mix the classes
df = df.sample(frac=1, random_state=42)

In [27]:
df.groupby('is_attack')['is_attack'].count()

is_attack
0    300000
1    285743
Name: is_attack, dtype: int64

In [28]:
df.groupby('traffic')['traffic'].count()

traffic
apachekiller      2334
arpspoofing      11222
camoverflow      62500
mqttmalaria       4961
netscan          62500
normal          300000
rudeadyet        71990
slowloris        63496
slowread          6740
Name: traffic, dtype: int64

---------------------------------------

## Create Model & Train Model

In [29]:
x_columns = df.columns.drop('is_attack')

le = LabelEncoder()
le.fit(df["is_attack"].values)

x = df[x_columns].values
y = df["is_attack"].values
y = le.transform(y)

In [30]:
# Split into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42, stratify=y)

In [31]:
x_test.shape,y_test.shape

((146436, 31), (146436,))

In [32]:
x_train.shape,y_train.shape

((439307, 31), (439307,))

----------

## Save Train and Test Dataset

In [33]:
# Convert arrays to Series or DataFrame
x_train = pd.DataFrame(x_train, columns=x_columns)
y_train = pd.Series(y_train, name='is_attack')
x_test = pd.DataFrame(x_test, columns=x_columns)
y_test = pd.Series(y_test, name='is_attack')

# Create new DataFrames for training and testing sets
train_df = pd.concat([x_train, y_train], axis=1)
test_df = pd.concat([x_test, y_test], axis=1)

# Directory to save the files
save_dir = '../../../Datasets/Flows/'

# Save train and test DataFrames to CSV files in the specified directory
train_path = os.path.join(save_dir, 'train.csv')
test_path = os.path.join(save_dir, 'test.csv')

train_df.to_csv(train_path, index=False)
test_df.to_csv(test_path, index=False)